In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import export_text
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [13]:
products = pd.read_csv("/Users/rishikeshdhokare/Documents/Ironhack/FinalProject/instacart-customer-churn-analysis/datafiles/products.csv")
order_products_train = pd.read_csv("/Users/rishikeshdhokare/Documents/Ironhack/FinalProject/instacart-customer-churn-analysis/datafiles/order_products__train.csv")
order_products_prior = pd.read_csv("/Users/rishikeshdhokare/Documents/Ironhack/FinalProject/instacart-market-basket-analysis/datafiles/order_products__prior.csv") 
departments = pd.read_csv("/Users/rishikeshdhokare/Documents/Ironhack/FinalProject/instacart-market-basket-analysis/datafiles/departments.csv")
aisles = pd.read_csv("/Users/rishikeshdhokare/Documents/Ironhack/FinalProject/instacart-market-basket-analysis/datafiles/aisles.csv")
orders = pd.read_csv("/Users/rishikeshdhokare/Documents/Ironhack/FinalProject/instacart-customer-churn-analysis/datafiles/orders.csv")

# What are the top 5 products

In [ ]:
import pandas as pd

# Load the necessary datasets
order_products_prior = pd.read_csv("/kaggle/input/order-products-prior/order_products__prior.csv")  # Path for prior orders
order_products_train = pd.read_csv("/kaggle/input/order-products-train/order_products__train.csv")  # Path for train orders
products = pd.read_csv("/kaggle/input/products/products.csv")  # Correct path to products dataset

# Combine the prior and train datasets
combined_orders = pd.concat([order_products_prior[['product_id', 'add_to_cart_order']],
                             order_products_train[['product_id', 'add_to_cart_order']]])

# Filter for products that are added to the cart first
first_added = combined_orders[combined_orders['add_to_cart_order'] == 1]

# Count occurrences of each product being the first added to the cart
first_added_counts = first_added['product_id'].value_counts().reset_index()
first_added_counts.columns = ['product_id', 'AddToCartCount']

# Merge with products to get product names
first_added_counts = first_added_counts.merge(products[['product_id', 'product_name']], on='product_id', how='left')

# Sort and select the top 5
top_first_added = first_added_counts.sort_values(by='AddToCartCount', ascending=False).head(5)

print(top_first_added[['product_id', 'product_name', 'AddToCartCount']])


## Top Users

In [ ]:
# Load the necessary datasets
orders = pd.read_csv("/kaggle/input/orders/orders.csv")  # Path for orders dataset
order_products_prior = pd.read_csv("/kaggle/input/order-products-prior/order_products__prior.csv")  # Path for prior orders

# Merge the orders and order_products_prior datasets to get product details for each order
order_details = pd.merge(order_products_prior, orders[['order_id', 'user_id']], on='order_id', how='left')

# Group by 'user_id' and count the number of products they have ordered
user_order_counts = order_details.groupby('user_id')['product_id'].count().reset_index()
user_order_counts.columns = ['user_id', 'total_products_ordered']

# Sort users by total number of products ordered (descending)
user_order_counts_sorted = user_order_counts.sort_values(by='total_products_ordered', ascending=False)

# Find active customers (those with the most recent orders)
active_users = orders[orders['eval_set'] == 'test']['user_id'].unique()  # Assuming 'test' is for active users

# Filter the users who have ordered the most products and are active
active_top_users = user_order_counts_sorted[user_order_counts_sorted['user_id'].isin(active_users)]

# Print the top active users who have ordered the most products
print(active_top_users.head())  # Display the top users who ordered the most products

In [ ]:
import pandas as pd
from itertools import combinations
from collections import Counter

# Path to the files
order_products_prior_path = "/kaggle/input/order-products-prior/order_products__prior.csv"
products_path = "/kaggle/input/products/products.csv"

# Load product names
products = pd.read_csv(products_path, dtype={'product_id': 'category'})

# Initialize a Counter for tracking product pairs
pair_counter = Counter()

# Process in chunks
chunk_size = 1_000_000
for chunk in pd.read_csv(order_products_prior_path, chunksize=chunk_size, dtype={'product_id': 'category', 'order_id': 'category'}):
    # Group by order_id and get unique product combinations
    grouped = chunk.groupby('order_id')['product_id'].apply(lambda x: list(combinations(sorted(x.unique()), 2)))
    for pairs in grouped:
        pair_counter.update(pairs)

# Convert counter to DataFrame
combination_counts = pd.DataFrame(pair_counter.items(), columns=['product_pair', 'combination_count'])
combination_counts[['product_id_1', 'product_id_2']] = pd.DataFrame(combination_counts['product_pair'].tolist(), index=combination_counts.index)
combination_counts.drop(columns=['product_pair'], inplace=True)

# Merge with product names
combination_counts = combination_counts.merge(products[['product_id', 'product_name']], left_on='product_id_1', right_on='product_id').drop('product_id', axis=1).rename(columns={'product_name': 'product_name_1'})
combination_counts = combination_counts.merge(products[['product_id', 'product_name']], left_on='product_id_2', right_on='product_id').drop('product_id', axis=1).rename(columns={'product_name': 'product_name_2'})

# Sort by combination count in descending order and get the top 5 combinations
top_combinations = combination_counts.sort_values(by='combination_count', ascending=False).head(5)

# Display results
print(top_combinations[['product_name_1', 'product_name_2', 'combination_count']])


In [ ]:
import pandas as pd

# Load the datasets with memory optimization
#order_products_prior = pd.read_csv("/kaggle/input/order-products-prior/order_products__prior.csv", dtype={'product_id': 'category', 'order_id': 'category'})
#products = pd.read_csv("/kaggle/input/products/products.csv", dtype={'product_id': 'category'})

# Merge the products data with order_products_prior to get product names
order_details = pd.merge(order_products_prior, products[['product_id', 'product_name']], on='product_id', how='left')
# Generate product combinations in each order
# Create a DataFrame where each row is a combination of two products from the same order
product_combinations = order_details.merge(order_details, on='order_id', suffixes=('_1', '_2'))

# Filter for unique product pairs (product_1 < product_2 to avoid duplication)
product_combinations = product_combinations[product_combinations['product_id_1'] < product_combinations['product_id_2']]

# Count the frequency of each product combination
combination_counts = product_combinations.groupby(['product_id_1', 'product_id_2']).size().reset_index(name='combination_count')

# Merge with product names to get the product names
combination_counts = combination_counts.merge(products[['product_id', 'product_name']], left_on='product_id_1', right_on='product_id', how='left').drop('product_id', axis=1).rename(columns={'product_name': 'product_name_1'})
combination_counts = combination_counts.merge(products[['product_id', 'product_name']], left_on='product_id_2', right_on='product_id', how='left').drop('product_id', axis=1).rename(columns={'product_name': 'product_name_2'})

# Sort by combination count in descending order and get the top 5 combinations
top_combinations = combination_counts.sort_values(by='combination_count', ascending=False).head(5)

# Display the top 5 most frequent product combinations
print(top_combinations[['product_name_1', 'product_name_2', 'combination_count']])


In [ ]:
import dask.dataframe as dd
from itertools import combinations
from collections import Counter
import pandas as pd

order_products_prior_path = "/Users/rishikeshdhokare/Documents/Ironhack/FinalProject/instacart-market-basket-analysis/datafiles/order_products__prior.csv"

# Load Dask DataFrame
order_products_prior = dd.read_csv(order_products_prior_path, dtype={'product_id': 'category', 'order_id': 'category'})

# Define a function to generate combinations and count them
def count_combinations(df):
    counter = Counter()
    for order, group in df.groupby('order_id'):
        products = sorted(group['product_id'].unique())
        counter.update(combinations(products, 2))
    return pd.DataFrame(counter.items(), columns=['product_pair', 'combination_count'])

# Apply function on each partition
combination_counts_ddf = order_products_prior.map_partitions(count_combinations).compute()

# Convert product pairs to separate columns
combination_counts_ddf[['product_id_1', 'product_id_2']] = pd.DataFrame(
    combination_counts_ddf['product_pair'].tolist(), index=combination_counts_ddf.index)
combination_counts_ddf.drop(columns=['product_pair'], inplace=True)

# Aggregate the combination counts
combination_counts_ddf = combination_counts_ddf.groupby(['product_id_1', 'product_id_2'])['combination_count'].sum().reset_index()

# Sort and retrieve top 5 combinations
top_combinations = combination_counts_ddf.sort_values(by='combination_count', ascending=False).head(5)

# Display results
print(top_combinations)


/var/folders/43/30gtvzf54299gghzdyc8_l900000gn/T/ipykernel_64667/3161782411.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for order, group in df.groupby('order_id'):
/var/folders/43/30gtvzf54299gghzdyc8_l900000gn/T/ipykernel_64667/3161782411.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for order, group in df.groupby('order_id'):
/var/folders/43/30gtvzf54299gghzdyc8_l900000gn/T/ipykernel_64667/3161782411.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to ad